In [ ]:
import os
import time
import json
import requests
import tempfile
import numpy as np
import pandas as pd

import boto3
import sagemaker
from sagemaker.inputs import TrainingInput, CreateModelInput
from sagemaker.workflow.steps import TrainingStep, CreateModelStep, ProcessingStep

from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.workflow.parameters import ParameterString
from sagemaker.model import Model
from sagemaker.pipeline import PipelineModel
from sagemaker.workflow.model_step import ModelStep
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingOutput, ProcessingInput

from sagemaker.sklearn import SKLearnModel

In [4]:
region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)
sagemaker_boto_client = boto_session.client("sagemaker")

# sagemaker_session = sagemaker.session.Session()
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_boto_client
)

account = sagemaker_session.boto_session.client("sts").get_caller_identity()["Account"]
# region = sagemaker_session.boto_session.region_name
s3_client = boto3.client("s3", region_name=region)
sm_client = boto3.client("sagemaker", region_name=region)

sm_role = get_execution_role()

In [6]:
# S3 prefix
bucket = "artwork-content-trial-bucket"
prefix="mch-artwork-content"
train_data_dir_prefix="data"
pipeline_dir_prefix="pipeline-data"
training_input_prefix = "content_ovr_anonymised.csv"
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="Approved"
)


In [99]:
training_input = f"s3://{bucket}/{train_data_dir_prefix}/{training_input_prefix}"

In [100]:
train_image_uri = '791574662255.dkr.ecr.us-east-1.amazonaws.com/artwork-content-repo:latest'
estimator_output_path = f"s3://{bucket}/{pipeline_dir_prefix}/"

In [101]:

content_estimator = Estimator(
    image_uri=train_image_uri,
    instance_type="ml.m5.xlarge",
    instance_count=1,
#     entry_point='inference.py',
    output_path=estimator_output_path,
    role=sm_role,
)

In [102]:
step_train = TrainingStep(
    name="training",
    estimator=content_estimator,
    inputs=training_input,
)

In [103]:
mdl_name="artwork_content_model"
# code_location = 's3://{0}/{1}/code'.format(bucket, prefix)

model = Model(
    image_uri=content_estimator.training_image_uri(),
    name=mdl_name,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=sm_role
)

In [104]:
print(model)

In [105]:
pipeline_session = PipelineSession()

pipeline_model = PipelineModel(
        models=[model],
        role=sm_role,
        sagemaker_session=pipeline_session,
)

In [106]:
# timestamp = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
# mpg_name = f"artwork-content-{timestamp}"
mpg_name = "MCH-Content-Models"

register_model_step_args = pipeline_model.register(
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=["ml.t2.medium", "ml.m5.xlarge"],
    model_package_group_name=mpg_name,
    approval_status=model_approval_status
)

In [107]:
model_name = "artwork-content-model" 
step_model_registration = ModelStep(
   name=model_name,
   step_args=register_model_step_args,
)

In [108]:
# step_model_create = ModelStep(
#    name=f"{model_name}-creation",
#    step_args=model.create(instance_type="ml.m5.xlarge"),
# )

In [109]:
# s3_client.upload_file(Filename="deploy.py", Bucket=bucket, Key="code/deploy.py")
# deploy_script_uri = f"s3://{bucket}/code/deploy.py"

# deployment_processor = SKLearnProcessor(
#     framework_version="1.0-1",
#     role=role,
#     instance_type="ml.t3.medium",
#     instance_count=1,
#     base_job_name=f"{prefix}-deploy",
#     sagemaker_session=sagemaker_session,
# )

# step_deploy_model = ProcessingStep(
#     name="DeployContentModel",
#     processor=deployment_processor,
#     job_arguments=[
#         "--model-name",
#         step_model_create.properties.ModelName,
#         "--region",
#         region,
#         "--endpoint-instance-type",
#         "ml.m5.xlarge",
#         "--endpoint-name",
#         "mch-artwork-content-endpoint-1",
#     ],
#     code=deploy_script_uri,
# )

In [110]:
# deployment_processor = SKLearnProcessor(
#     framework_version="1.0-1",
#     role=role,
#     instance_type="ml.t3.medium",
#     instance_count=1,
#     base_job_name=f"{prefix}-deploy",
#     sagemaker_session=sagemaker_session,
# )
    
# step_deploy_model = ProcessingStep(
#     name="DeployContentModel",
#     processor=deployment_processor,
#     job_arguments=[
#         "--arn-path",
#         step_model_create.properties.ModelArn,
#         "--region",
#         region
#     ],
#     code='deploy_2.py',
#     outputs=[
#         ProcessingOutput(output_name="endpoint_arn", source="/opt/ml/processing/endpoint_arn")
#     ],
# )

In [111]:
# endpoint_name = "mch-content-ep"
# initial_instance_count=1
# instance_type='ml.t2.medium'

# step_deploy = pipeline_model.deploy(initial_instance_count=1,instance_type='ml.t2.medium', endpoint_name=endpoint_name)
# step_deploy = model.deploy(initial_instance_count=initial_instance_count, instance_type=instance_type, endpoint_name=endpoint_name)

In [112]:
pipeline_name = f"artwork-content-pipeline-demo"

pipeline_steps = [step_train, step_model_registration]

pipeline = Pipeline(
    name=pipeline_name,
    parameters=[model_approval_status],
    steps=pipeline_steps,
)

In [113]:
# json.loads(pipeline.definition())

In [114]:
pipeline.upsert(role_arn=sm_role)

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:791574662255:pipeline/artwork-content-pipeline-demo',
 'ResponseMetadata': {'RequestId': 'd9641067-e42b-4b9c-b32a-c023e38d15c6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd9641067-e42b-4b9c-b32a-c023e38d15c6',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '97',
   'date': 'Tue, 10 Jan 2023 09:05:45 GMT'},
  'RetryAttempts': 0}}

In [115]:
execution = pipeline.start()
execution.wait()
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:us-east-1:791574662255:pipeline/artwork-content-pipeline-demo',
 'PipelineExecutionArn': 'arn:aws:sagemaker:us-east-1:791574662255:pipeline/artwork-content-pipeline-demo/execution/vn6z7ecsnduw',
 'PipelineExecutionDisplayName': 'execution-1673341545392',
 'PipelineExecutionStatus': 'Succeeded',
 'PipelineExperimentConfig': {'ExperimentName': 'artwork-content-pipeline-demo',
  'TrialName': 'vn6z7ecsnduw'},
 'CreationTime': datetime.datetime(2023, 1, 10, 9, 5, 45, 300000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2023, 1, 10, 9, 9, 35, 671000, tzinfo=tzlocal()),
 'CreatedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:791574662255:user-profile/d-u9ximrvnxnyj/vchitrakathi-dminc-com-f51',
  'UserProfileName': 'vchitrakathi-dminc-com-f51',
  'DomainId': 'd-u9ximrvnxnyj'},
 'LastModifiedBy': {'UserProfileArn': 'arn:aws:sagemaker:us-east-1:791574662255:user-profile/d-u9ximrvnxnyj/vchitrakathi-dminc-com-f51',
  'UserProfileName': 'vchitra

In [ ]:
import time
from sagemaker.lineage.visualizer import LineageTableVisualizer


viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

In [134]:
# import os

# os.path.realpath('scoring/')

In [167]:
# ModelName='artwork_content_model'
# ModelName[:20]

In [34]:
BASE_DIR = os.path.realpath('.'))
BASE_DIR

'/root'

In [29]:
# model_name = 'pipelines-j84lf8hkcds5-CreateModelStep-WpoTpbCPDJ'
# model_name = 'pipelines-uhxcwx86vgv4-CreateModelStep-RQ6PyNYjFk'
# models = sm_client.list_models()
# models
# model_name = models['Models'][0]['ModelName']
# model_name

# response = sm_client.describe_model(ModelName=model_name)
# response

# model_data_url = response['PrimaryContainer']['ModelDataUrl']
# model_data_url

mpg_name='MCH-Content-Models'
model_package_arn = sm_client.list_model_packages(ModelPackageGroupName=mpg_name)['ModelPackageSummaryList'][0]['ModelPackageArn']
model_package_arn

'arn:aws:sagemaker:us-east-1:791574662255:model-package/mch-content-models/85'

In [15]:
# sagemaker_session.list_s3_files(bucket=bucket, key_prefix='code/')

In [81]:
# role = boto3.client("iam")
# role.get_role(RoleName="AmazonSageMakerServiceCatalogProductsExecutionRole")

# access_key = boto3.Session().get_credentials().access_key
# secret_key = boto3.Session().get_credentials().secret_key
# token = boto3.Session().get_credentials().token

# print(access_key, "\n\t", secret_key, "\n\t\t", token)

In [109]:
# from sagemaker.sklearn.processing import SKLearnProcessor
# SKLearnProcessor.run()


# from sagemaker.workflow.steps import ProcessingStep
# ProcessingStep()

In [3]:
# import pandas as pd

# model_df = pd.read_excel('aws-models.xlsx')

# model_df.head()

# model_names_lst = model_df['Name'].to_list()


# for name in model_names_lst:
#     # Delete model
#     sm_client.delete_model(ModelName=name)

In [ ]:
# response = sm.list_pipeline_executions(PipelineName=pipeline_name)

# # Get the ARN of the latest pipeline execution
# latest_pipeline_execution_arn = response['PipelineExecutionSummaries'][0]['PipelineExecutionArn']

# # Get the response of latest execution of the pipeline
# pipeline_arn_response = sm.describe_pipeline_execution(
#     PipelineExecutionArn=latest_pipeline_execution_arn
# )
